### Перепишем код с урока и раздел решения домашки поместим вниз:

In [99]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
!pip install pyLDavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [102]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize
from gensim.corpora.dictionary import Dictionary
import pymorphy2  
import nltk 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

%matplotlib inline
pd.set_option("display.max_rows", 6)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/text_less_2/materials.csv', "rb") as f:
    news = pd.read_csv(f)

print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [104]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/text_less_2/users_articles.csv', "rb") as f2:
    users = pd.read_csv(f2)
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [105]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [106]:
stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

In [107]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/text_less_2/stopwords.txt') as f3:
    additional_stopwords = [w.strip() for w in f3.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [108]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация             c помощью библиотеки pymorphy2 (в else)
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    #print(tokens)
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form   # здесь включается морфоанализатор и меняет форму слов на неопределенную изменяя окончания
                words_lem.append(temp_cach)
                #print(w, ':', temp_cach)
    

    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    #print(words_lem_without_stopwords)
    return words_lem_without_stopwords

In [109]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: user 33.9 s, sys: 1.31 s, total: 35.2 s
Wall time: 40.4 s


In [110]:
%%time
#Запускаем лемматизацию текста. Будет очень долго... На выходе получаем в рамках каждой статьи список разбитых леммитизированный слов
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 4min 20s, sys: 843 ms, total: 4min 21s
Wall time: 4min 22s


In [111]:
texts = [t for t in news['title'].values]

common_dictionary = Dictionary(texts)    #словарь indx: words
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [112]:
n_topic = 20

In [113]:
from gensim.models import LdaModel

lda = LdaModel(common_corpus, num_topics=n_topic, id2word=common_dictionary)

In [114]:
def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector1 = np.mean(user_vector, 0)  # можно не средний
    return user_vector1 #, user_vector

In [115]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(n_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [116]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(n_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(n_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.059365,0.0,0.000000,0.0,0.0,0.000000,0.0,0.284383,0.172586,...,0.000000,0.476649,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,4896,0.314345,0.0,0.573763,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.091160,0.0,0.0,0.000000,0.0
2,4897,0.704228,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.144117,0.000000,0.0,0.0,0.0,0.130404,0.0,0.0,0.000000,0.0
3,4898,0.272811,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.667290,0.0,0.0,0.049534,0.0
4,4899,0.119753,0.0,0.000000,0.0,0.0,0.697576,0.0,0.122491,0.000000,...,0.038558,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [117]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(n_topic)]].values))

In [118]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(n_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(n_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.089314,0.027457,0.013793,0.144445,0.056491,0.073310,0.062638,0.022579,0.062760,...,0.027319,0.014997,0.022454,0.108253,0.032407,0.064618,0.009696,0.029635,0.090682,0.016257
1,u108690,0.020655,0.016697,0.011501,0.140768,0.004919,0.058207,0.040718,0.052780,0.151403,...,0.004554,0.076041,0.010990,0.038328,0.079806,0.094386,0.044812,0.008209,0.095995,0.011273
2,u108339,0.008296,0.058777,0.065263,0.013407,0.025135,0.025373,0.013991,0.118384,0.173701,...,0.008453,0.067102,0.000000,0.061375,0.202574,0.077660,0.002089,0.010963,0.036600,0.019953


In [119]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/text_less_2/users_churn.csv') as f4:
    target = pd.read_csv(f4)

In [120]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.089314,0.027457,0.013793,0.144445,0.056491,0.073310,0.062638,0.022579,0.062760,...,0.014997,0.022454,0.108253,0.032407,0.064618,0.009696,0.029635,0.090682,0.016257,0
1,u108690,0.020655,0.016697,0.011501,0.140768,0.004919,0.058207,0.040718,0.052780,0.151403,...,0.076041,0.010990,0.038328,0.079806,0.094386,0.044812,0.008209,0.095995,0.011273,1
2,u108339,0.008296,0.058777,0.065263,0.013407,0.025135,0.025373,0.013991,0.118384,0.173701,...,0.067102,0.000000,0.061375,0.202574,0.077660,0.002089,0.010963,0.036600,0.019953,1


In [121]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(n_topic)]], 
                                                    X['churn'], random_state=0)

In [122]:
logreg = LogisticRegression().fit(X_train, y_train)

In [123]:
preds = logreg.predict_proba(X_test)[:, 1]

In [124]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [125]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
auc = roc_auc_score(y_test, preds)  #  добавил метрику аук 

ix = np.argmax(fscore)
print('Best Threshold=%f, roc_auc=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        auc,
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.263803, roc_auc=0.932, F-Score=0.638, Precision=0.610, Recall=0.669


### Решение домашки:

1.  Самостоятельно разобраться с tfidf 

tfidf- это перемножение частотности слова в одном документе/статье (tf) на обратную частотность слова во всех документах/статьях (idf)



2) Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

__Решение:__

In [126]:
from sklearn.metrics import roc_auc_score

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector1 = np.median(user_vector, 0)  # заменили mean на median
    return user_vector1 

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(n_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(n_topic)]]

X = pd.merge(user_embeddings, target, 'left')

X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(n_topic)]], 
                                                    X['churn'], random_state=0)

logreg_median = LogisticRegression().fit(X_train, y_train)
preds_median = logreg_median.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds_median)
fscore = (2 * precision * recall) / (precision + recall)
auc = roc_auc_score(y_test, preds_median )

ix = np.argmax(fscore)
print('Best Threshold=%f, roc_auc=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        auc,
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.277260, roc_auc=0.963, F-Score=0.744, Precision=0.714, Recall=0.776


Наблюдаем что все метрики улучшились, порог понизился.

3) Сделать то же самое, но с max.

__Решение:__

In [127]:
def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector1 = np.max(user_vector, 0)  # заменили  median на max
    return user_vector1 

user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(n_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(n_topic)]]

X = pd.merge(user_embeddings, target, 'left')

X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(n_topic)]], 
                                                    X['churn'], random_state=0)

logreg_max = LogisticRegression().fit(X_train, y_train)
preds_max = logreg_max.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds_max)
fscore = (2 * precision * recall) / (precision + recall)
auc = roc_auc_score(y_test, preds_max )

ix = np.argmax(fscore)
print('Best Threshold=%f, roc_auc=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        auc,
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.334950, roc_auc=0.958, F-Score=0.737, Precision=0.709, Recall=0.767


Видим что с median рок аук и ф мера лучше всех остальных вариантов.


4) (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.


__Решение:__

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

with open('/content/drive/MyDrive/ГБ/выборки для исследований/text_less_2/materials.csv', "rb") as f5:
    data = pd.read_csv(f5)

In [129]:
data['title'][1]

'Матч 1/16 финала Кубка России по футболу был приостановлен судьей из-за взрыва пиротехнических снарядов, передает корреспондент «Газеты.Ru». Болельщики выбросили на поле петарды. Судья увел команды с поля в подтрибунное помещение. Динамовцы ушли, а торпедовцы остались у кромки поля. Сообщается, что матч остановлен на пять минут.n«Газета.Ru» ведетnонлайн-трансляциюnэтого матча.'

Запишем новые столбец в датафрейм со статьями, где будут статьи из очищенных и леммитированных слов

In [130]:

%%time

lst = []
for i in range(len(data.title)):
  lst.append(' '.join(lemmatization(data['title'][i])))

data['clean_title'] = lst

data.head(5)

CPU times: user 5min 10s, sys: 1.03 s, total: 5min 11s
Wall time: 5min 14s


,doc_id,title,clean_title
0,6,Заместитель председателяnправительства РФnСерг...,заместитель председатель правительство рф серг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...,матч 1/16 финал кубок россия футбол приостанов...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...,форвард авангард томаш заборский прокомментиро...
3,4898,Главный тренер «Кубани» Юрий Красножан прокомм...,главный тренер кубань юрий красножанин прокомм...
4,4899,Решением попечительского совета владивостокско...,решение попечительский совет владивостокский с...


Знаю как посмотреть список важности слов по idf, но теперь не понял как соотнести статьи tfidf и модель МО

In [131]:
#tf0 = TfidfVectorizer(stop_words=stopword_ru, max_features=20)

#С помощью параметра min_df мы можем задать минимальное количество документов, в котором должен появиться токен.
tf0 = TfidfVectorizer(min_df=15, norm=None)
tf0.fit(data['clean_title'])

TfidfVectorizer(min_df=15, norm=None)

In [132]:
df_idf = pd.DataFrame(tf0.idf_, index=tf0.get_feature_names_out(), columns=['idf_weights'])

df_idf.sort_values(by=['idf_weights'])

,idf_weights
россия,1.791001
всё,1.993489
nn,1.999609
...,...
про,8.431040
277,8.431040
недорого,8.431040




5) Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

__Решение:__

In [167]:
def get_user_embedding(user_articles_list, mode):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector1 = mode(user_vector, 0)  # заменили  median на max
    return user_vector1 

mods = [['mean  ', np.mean], ['max   ', np.max], ['median', np.median]]
differents = pd.DataFrame()

for mode in mods:
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, mode[1]), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(n_topic)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(n_topic)]]

    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(n_topic)]], 
                                                        X['churn'], random_state=0)

    logreg_max = LogisticRegression().fit(X_train, y_train)
    preds_max = logreg_max.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds_max)
    fscore = (2 * precision * recall) / (precision + recall)
    auc = roc_auc_score(y_test, preds_max )

    ix = np.argmax(fscore)
    new_row = pd.DataFrame({'roc_auc': [auc], 'F-Score': [fscore[ix]], 'Precision': [precision[ix]], 'Recall':[recall[ix]]}, index=[mode[0]])
    print(new_row) # вывел принт, т.к. датафрейм не записывается... 

    differents.append(new_row, ignore_index=True) # Не понял почему не вставляется строка
    
differents  # печатает пустой датафрейм

         roc_auc   F-Score  Precision    Recall
mean    0.931891  0.638132   0.609665  0.669388
         roc_auc   F-Score  Precision    Recall
max     0.958393  0.737255   0.709434  0.767347
         roc_auc  F-Score  Precision   Recall
median  0.962537  0.74364   0.714286  0.77551


""


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных